# POSEIDON: Pose Estimation & Activity Recognition using GNNs

Team Members (Group 16): 
1. Chong Jun Rong Brian (A0290882U)
2. Parashara Ramesh (A0285647M)
3. Ng Wei Jie Brandon (A0184893L)

In [ ]:
%load_ext autoreload
%autoreload 2

NOTE: Need to add anchor jump links to each heading

<h2><u> Table of contents </u></h2>

1. What is this project about?
<br> 1.1. Project Motivation
<br> 1.2. Project Description
<br> 1.3. Project Setup 
<br> 1.4. Project Changes from Initial Proposal
<br> 1.5. Project Presentation Video

2. Model building approach
<br> 2.1. SimplePose (Baseline model) 
<br> 2.2. SimplePoseGNN
<br> 2.3. SimplePoseGAT
<br> 2.4. SimplePoseTAG

3. Human 3.6M Dataset
<br> 3.1. Summary of the dataset
<br> 3.2. Preparing the dataset
<br> 3.3. Visualizing poses
<br> 3.4. Training baseline model (SimplePose)
<br> 3.5. Training Graph Convolutional model (SimplePoseGNN)
<br> 3.6. Training Graph Transformer model (SimplePoseGAT)
<br> 3.7. Training Toplogy Adaptive Graph Convolutional model (SimplePoseTAG)

4. Custom Dataset
<br> 4.1. Summary of the dataset
<br> 4.2. Preparing the dataset
<br> 4.3. Visualizing poses
<br> 4.4. Training baseline model (SimplePose)
<br> 4.5. Training Graph Convolutional model (SimplePoseGNN)
<br> 4.6. Training Graph Transformer model (SimplePoseGAT)
<br> 4.7. Training Toplogy Adaptive Graph Convolutional model (SimplePoseTAG)

5. Evaluation
<br> 5.1. Human 3.6M dataset
<br> 5.2. Custom Dataset

6. Lessons Learnt & Conclusions
7. Resources


<h2><u>1. What is this project about?</u></h2>
<h3><u>1.1. Project Motivation</u></h3>

Accurately predicting 3D human poses from 2D keypoints is essential for applications like motion capture and activity recognition. Traditional methods, such as direct regression or lifting techniques, often fail to capture the intricate spatial relationships between body joints. By modeling 2D pose keypoints as graphs, we can utilize the inherent connectivity between joints to enhance 3D pose estimation. Furthermore, recognizing and classifying human activities from these poses is vital in areas like surveillance and healthcare. This project aims to investigate how Graph Neural Networks (GNNs) can improve both 3D pose estimation and activity recognition.

<h3><u>1.2. Project Description</h3></u>

This project has two primary objectives.

The first is to accurately predict 3D human poses from 2D keypoints using Graph Neural Networks (GNNs).

* We will begin by developing two baseline models: one utilizing a standard Neural Network (NN) and Convolutional Neural Network (CNN), and the other employing a simple GNN for 3D pose estimation.
* Next, we will reimplement the SemGCN model, which represents 2D pose keypoints as graph nodes with edges capturing the connectivity between joints.
* Finally, we aim to enhance the SemGCN model by exploring alternative GNN architectures and introducing modifications to improve its performance.

The second objective is to classify human activities based on 2D keypoints. For this task, we will use custom datasets to evaluate the generalization capabilities of GNN-based models for activity recognition.

<h3><u>1.3. Project Setup</u></h3>

(TODO need to rewrite this part)

1. Install the dependencies from `requirements.txt` using the command

`pip install -r requirements.txt`

<h3><u>1.4. Project Changes from Initial Proposal</u></h3>

TODO

<h3><u>1.5. Project Presentation Video</u></h3>

(TODO add our presentation slides as well)

Our project presentation has been uploaded to youtube and can be found at <a href="https://youtu.be/O3Qb1UfjszM">this link</a>. 

<h2><u>2. Model Building Approach</u></h2>

Previous works have attempted to do pose estimation and activity recognition via normal Neural Network methods such as Convolutional Neural Networks such as [PoseCNN: A Convolutional Neural Network for 6D Object Pose Estimation in Cluttered Scenes by Xiang. et al](https://arxiv.org/abs/1711.00199) and [Skeleton-based Human Action Recognition via
Convolutional Neural Networks (CNN) by Ali. et al](https://arxiv.org/pdf/2301.13360).



<h3><u>2.1 SimplePose (Baseline model)</u></h3>

To study the effectiveness of Graph Neural Networks in 3D Pose Estimation and Activity Recognition, we built our own basic Neural Network model which we will call it as SimplePose.

SimplePose is a simple Neural Network model that uses the same model architecture as [A simple yet effective baseline for 3d human pose estimation by Martinez. et al](https://arxiv.org/abs/1705.03098) but with a few tweaks compared against the original architecture.

These are the tweaks that we have done for SimplePose:-

1. We increased the dropout rate from 0.5 to 0.6 to ensure stable training and to prevent overfitting.
2. Instead of having 2 blocks as shown in Figure 1, we have 6 blocks because we found out it having bigger models enabled the multitask learning to perform well for SimplePose.
3. After the last block, we add a small Neural Network for 3D Pose Estimation and another small Neural Network for Activity Recognition. Both of their inputs is the last block final output.

You can see the SimplePose's implementation at [models/simple_pose.py](./models/simple_pose.py)

The figure below illustrates the SimplePose Architecture:-

<img title="SimplePose" alt="SimplePose" src="./notebook_images/SimplePose.drawio.png">


Below shows the number of trainable parameters for SimplePose:-

In [ ]:
from models.simple_pose import SimplePose
from utils.pytorch_utils import count_parameters
model = SimplePose(total_joints=13, total_actions=5)
parameters = count_parameters(model)
print(f"SimplePose Trainable Parameters: {parameters}")

<h3><u>2.2 SimplePoseGNN</u></h3>

We built 3 Graph Neural Networks to compare against SimplePose. Our first model is a Graph Convolutional Neural Network which we will call it as SimplePoseGNN.

SimplePoseGNN is a Graph Convolutional Neural Network model that was proposed by [Kipf. et al](https://arxiv.org/pdf/1609.02907v4). We decided to implement this model because it is the baseline model for Graph Neural Networks.

For our Graph Convolutional Layers for SimplePoseGNN, we used [GraphConv from DGL](https://docs.dgl.ai/en/1.1.x/generated/dgl.nn.pytorch.conv.GraphConv.html) in its default settings to build our own GraphConvModule.

A GraphConvModule consists of these layers in sequence.:-
1. GraphConv
2. BatchNorm1D
3. ReLU
4. GraphConv
5. BatchNorm1D
6. Dropout 
7. ReLU
8. Linear Layer
   
There is also a residual connection that connects from the input of the first GraphConv with the output from Linear Layer.

SimplePoseGNN has 6 GraphConvModule to ensure good performance.

Not only that, these are the additional modifications we have done for SimplePoseGNN:-

1. We compute the Laplacian Positional Encoding of our graph inputs using [dgl.transforms.LapPE](https://docs.dgl.ai/en/1.1.x/generated/dgl.transforms.LapPE.html) with k = 5.
2. We embed our Node Features, Laplacian Positional Encodings and Edge features into their own nn.Linear layers and concatenate the values before giving it to the first GraphConvModule.
3. We add a small Neural Network for 3D Pose Estimation and Activity Recognition after the last GraphConvModule. Their inputs are the output of the last GraphConvModule.

You can see the SimplePoseGNN's implementation at [models/simple_pose_gnn.py](./models/simple_pose_gnn.py)

Here is SimplePoseGNN's architecture as described:-

<img title="SimplePoseGNN" alt="SimplePoseGNN" src="./notebook_images/SimplePoseGNN.drawio.png">

Below shows the number of trainable parameters for SimplePoseGNN:-

In [ ]:
from models.simple_pose_gnn import SimplePoseGNN
from utils.pytorch_utils import count_parameters
model = SimplePoseGNN(hidden_size=80, num_classes=5)
parameters = count_parameters(model)
print(f"SimplePoseGNN Trainable Parameters: {parameters}")

<h3><u>2.3 SimplePoseGAT</u></h3>

Our 2nd Graph Neural Network implementation is the Graph Transformer Neural Network which we denote it as SimplePoseGAT.

SimplePoseGAT is a Graph Transformer Neural Network model that is proposed by [Dwivedi et al](https://arxiv.org/pdf/2012.09699v2). We decided to implement this model because we want to investigate if having attention scores can help improve the tasks' performance compared against all the other models.

For this implementation, we followed the Graph Transformer's implementation as described in [DGL's Graph Transformers Tutorial](https://docs.dgl.ai/en/1.1.x/notebooks/sparse/graph_transformer.html) but with our own tweaks.

These are the tweaks that we have done for SimplePoseGAT:-

1. We compute the Laplacian Positional Encoding of our graph inputs using [dgl.transforms.LapPE](https://docs.dgl.ai/en/1.1.x/generated/dgl.transforms.LapPE.html) with k = 20.
2. We embed our Node Features, Laplacian Positional Encodings and Edge features into their own nn.Linear layers and concatenate the values before giving it to the MultiHeadAttention Layer.
3. We reduced the Graph Transformer Layers to 2 due to GPU memory limitations.
4. We add a linear layer after the Graph Transformer to perform 3D Pose Estimations.
5. We add a SumPooling layer followed by a small neural network model to perform Activity Recognition.

You can see the SimplePoseGAT's implementation at [models/simple_pose_gat.py](./models/simple_pose_gat.py)

The figure below illustrates the SimplePoseGAT Architecture:-

<img title="SimplePoseGAT" alt="SimplePoseGAT" src="./notebook_images/SimplePoseGAT.drawio.png">


Below shows the number of trainable parameters for SimplePoseGAT:-

In [ ]:
from models.simple_pose_gat import SimplePoseGAT
from utils.pytorch_utils import count_parameters
model = SimplePoseGAT(in_size=2, out_size=3, num_classes=5)
parameters = count_parameters(model)
print(f"SimplePoseGAT Trainable Parameters: {parameters}")

<h3><u>2.4 SimplePoseTAG</u></h3>

For our final Graph Neural Network model, we built a Topology Adaptive Graph Convolutional Neural Network that was proposed by [Du. et al](https://arxiv.org/pdf/1710.10370).

We will call this model as SimplePoseTAG. We decided to implement SimplePoseTAG because we want to see if allowing the model to interpret the topology of a graph can improve the tasks' performance as compared with other models

For this implementation. we followed the same approach as SimplePoseGNN as stated in Section 2.2 with a few differences.

1. We replaced the GraphConv layers with [TAGConv from DGL.](https://docs.dgl.ai/en/0.8.x/generated/dgl.nn.pytorch.conv.TAGConv.html). We also renamed this from GraphConvModule to TAGConvModule.

You can see the SimplePoseTAG's implementation at [models/simple_pose_tag.py](./models/simple_pose_tag.py)

The figure below illustrates the SimplePoseTAG Architecture:-

<img title="SimplePoseTAG" alt="SimplePoseTAG" src="./notebook_images/SimplePoseTAG.drawio.png">


Below shows the number of trainable parameters for SimplePoseTAG:-

In [ ]:
from models.simple_pose_tag import SimplePoseTAG
from utils.pytorch_utils import count_parameters
model = SimplePoseTAG(hidden_size=80, num_classes=5)
parameters = count_parameters(model)
print(f"SimplePoseTAG Trainable Parameters: {parameters}")

<h2><u>3. Human 3.6M Dataset</u></h2>

<h3><u>3.1. Summary of the dataset </u></h3>

The [Human 3.6M dataset](http://vision.imar.ro/human3.6m/description.php) is a comprehensive dataset comprising 3.6 million 3D human poses, captured from 11 professional actors performing 17 everyday activities (e.g., talking, smoking, discussing).

This dataset utilizes a 3D motion capture system with 10 cameras to track reflective markers placed on the body, enabling automatic labeling of joint positions.

Participants were provided with detailed task instructions and visual examples of the actions to perform, but they were also allowed some freedom to move naturally during execution.

As a result, the Human 3.6M dataset serves as a valuable resource for 3D human pose estimation, action recognition, and other computer vision research tasks.

<h3><u>3.2. Dataset preparation </u></h3>

Downloading the Human 3.6M dataset directly from the official website requires a login. To simplify access, we will use a preprocessed version of the dataset, which is available on Google Drive [here](https://drive.google.com/file/d/1JGt3j9q5A8WzUY-QKfyMVJUUvfreri-s/view?usp=drive_link).

The preprocessing was originally performed by [Martinez et al](https://github.com/una-dinosauria/3d-pose-baseline) and is sourced from their repository.

The following cells outline the steps to download this preprocessed dataset and create train-test files after applying necessary transformations.

From Section 3 (Models) onward, we will use the preprocessed train-test files directly. Therefore, executing the cells in this section is optional.

#### Downloading the zip from google drive

In [ ]:
# Download the zip from google drive
import gdown
import os

# file id from gdrive (refer to markdown cell above for full link)
file_id = '1JGt3j9q5A8WzUY-QKfyMVJUUvfreri-s' 
download_url = f'https://drive.google.com/uc?id={file_id}'

# Folder where you want to save the file
folder_path = os.path.join(os.getcwd(),"datasets", "h36m", "Original")
zip_file_path = os.path.join(folder_path, 'h36m.zip')

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Download the ZIP file
gdown.download(download_url, zip_file_path, quiet=False)

print(f"Downloaded ZIP file and saved to {zip_file_path}")


#### Extracting the contents into the same folder

In [ ]:
# extracting the contents inside the /datasets/h36m/Original folder
import zipfile

print(f"Extracting files to {folder_path}..")
# Open the ZIP file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)

print(f"Finished extracting files to {folder_path}")



#### Saving the 3d positions into a compressed np file

In [ ]:
import h5py
from glob import glob
from tqdm import tqdm
import numpy as np

output_filename = os.path.join(folder_path, 'data_3d_h36m')
subjects = ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11']

output = dict()
for subject in tqdm(subjects, desc= f'Processing subjects..'):
    output[subject] = dict()
    file_list = glob(os.path.join(folder_path, 'h36m', subject, 'MyPoses', '3D_positions', '*.h5'))
    assert len(file_list) == 30, "Expected 30 files for subject " + subject + ", got " + str(len(file_list))
    for f in file_list:
        action = os.path.splitext(os.path.basename(f))[0]

        if subject == 'S11' and action == 'Directions':
            continue  # Discard corrupted video

        with h5py.File(f) as hf:
            positions = hf.get('3D_positions')[:].reshape(32, 3, -1).transpose(2, 0, 1)
            positions /= 1000  # Meters instead of millimeters
            output[subject][action] = positions.astype('float32')

print(f'Saving compressed 3d positions into {output_filename}')
np.savez_compressed(output_filename, positions_3d=output)
print(f'Finished saving 3d positions into {output_filename}')
del output


#### Compute the ground truth 2d poses.

The motion capture system records 3D poses using 10 cameras in the world coordinate frame. By applying the intrinsic and extrinsic matrices of each camera, these 3D poses are projected into 2D poses as they would appear in the respective camera's frame. The coordinates are then normalized so that the x, y, and z values range between -1 and 1.

In [ ]:
from data.camera import world_to_camera, project_to_2d, image_coordinates, wrap
from data.h36m_dataset import Human36mDataset

print('Computing ground-truth 2D poses...')
output_filename_2d = os.path.join(folder_path, 'data_2d_h36m_gt')

dataset = Human36mDataset(output_filename + '.npz')
output_2d_poses = {}
for subject in dataset.subjects():
    output_2d_poses[subject] = {}
    for action in dataset[subject].keys():
        anim = dataset[subject][action]

        positions_2d = []
        for cam in anim['cameras']:
            pos_3d = world_to_camera(anim['positions'], R=cam['orientation'], t=cam['translation'])
            pos_2d = wrap(project_to_2d, True, pos_3d, cam['intrinsic'])
            pos_2d_pixel_space = image_coordinates(pos_2d, w=cam['res_w'], h=cam['res_h'])
            positions_2d.append(pos_2d_pixel_space.astype('float32'))
        output_2d_poses[subject][action] = positions_2d

print(f'Saving compressed 2d positions into {output_filename_2d}')
metadata = {
    'num_joints': dataset.skeleton().num_joints(),
    'keypoints_symmetry': [dataset.skeleton().joints_left(), dataset.skeleton().joints_right()]
}
np.savez_compressed(output_filename_2d, positions_2d=output_2d_poses, metadata=metadata)

print(f'Done saving compressed 2d positions into {output_filename_2d}')
del output_2d_poses

#### Cleanup the extracted files and the downloaded zip

In [ ]:
from shutil import rmtree

rmtree(os.path.join(folder_path, 'h36m'))
os.remove(os.path.join(folder_path, 'h36m.zip'))

#### Creating and saving train and test files from the saved npz files

In [ ]:
from utils.data_utils import read_3d_data, create_2d_data, create_train_test_files
from data.h36m_dataset import TRAIN_SUBJECTS, TEST_SUBJECTS, Human36mDataset
import os

subjects_train = TRAIN_SUBJECTS
subjects_test = TEST_SUBJECTS

processed_dataset_path = os.path.join(os.getcwd(), "datasets", "h36m", "Processed")
os.makedirs(processed_dataset_path, exist_ok=True)

dataset_path = os.path.join(os.getcwd(), "datasets", "h36m", "Original", "data_3d_h36m.npz")
dataset = Human36mDataset(dataset_path)


#### Normalizing the 2D poses to [-1, 1] while preserving aspect ratio and 3D poses to camera's frame

In [ ]:
print("Reading 3d npz file")
dataset = read_3d_data(dataset)

print("Reading 2d npz file")
dataset_2d_path = os.path.join(os.getcwd(), "datasets", "h36m", "Original", "data_2d_h36m_gt.npz")
keypoints = create_2d_data(dataset_2d_path, dataset)
print("Done")

#### Select subset of distinct actions to train and test models

In [ ]:
keep_actions = [
    'Greeting',
    'Photo',
    'Sitting',
    'SittingDown',
    'Walking'
]

#### Save the 2D poses, 3D poses, and actions in numpy files

In [ ]:
print("Creating train datasets and saving into file")
_, _, _ = create_train_test_files(subjects_train, dataset, keypoints, "train", processed_dataset_path, keep_actions)

print("Creating test datasets and saving into file")
_, _, _ = create_train_test_files(subjects_test, dataset, keypoints, "test", processed_dataset_path, keep_actions)
print("Done")

<h3><u>3.3 Visualizing poses </u></h3>

Since the train and test files have already been saved, we will use these files for the remainder of the notebook.

These files are also available in this [Google Drive Link](https://drive.google.com/drive/folders/1d38hWSM8clZlI11nqljxBECog1KY-iTd?usp=sharing), which can be placed in the local path to skip the execution of earlier cells.

The visualizations below indicate that, after the processing steps outlined above, human poses are represented as skeletons consisting of 13 joints.

Let's visualise the poses for the 5 actions in the h3.6m dataset

In [ ]:
from utils.visualization_utils import visualize_2d_pose_actions

out_poses_2d = np.load("datasets/h36m/Processed/test_2d_poses.npy")
out_actions = np.load("datasets/h36m/Processed/test_actions.npy")
for action in np.unique(out_actions):
    out_poses_2d_action = out_poses_2d[out_actions == action]
    visualize_2d_pose_actions(out_poses_2d_action, action=action)

Let's visualise the first 2D pose and corresponding 3D poses in the h3.6m dataset

In [ ]:
import numpy as np

from utils.visualization_utils import visualize_2d_pose, visualize_3d_pose

poses_2d = np.load("datasets/h36m/Processed/test_2d_poses.npy")
poses_3d = np.load("datasets/h36m/Processed/test_3d_poses.npy")

visualize_2d_pose(poses_2d[0])
visualize_3d_pose(poses_3d[0], elev=110, azim=90)

Let's visualise the distribution of the 5 actions in the h3.6m train and test datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def show_action_stats(actions, type):
    train_actions_df = pd.DataFrame(actions, columns=['actions'])
    action_counts = train_actions_df['actions'].value_counts()
    print(action_counts)
    
    action_counts.plot(kind='bar')
    
    # Step 4: Show the plot
    plt.title(f'{type} Action Distribution')
    plt.xlabel('Action')
    plt.ylabel('Count')
    plt.show()


In [ ]:
train_actions = np.load("datasets/h36m/Processed/train_actions.npy")
show_action_stats(train_actions, "Train")

In [ ]:
test_actions = np.load("datasets/h36m/Processed/test_actions.npy")
show_action_stats(test_actions, "Test")

In [ ]:
from dataloader.h36m_graph_loader_with_edge_feats import Human36MGraphEdgeDataset
import networkx as nx
import dgl

item = {
    'training_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'h36m', 'Processed', 'train_actions.npy'),
}

training_data = Human36MGraphEdgeDataset(item['training_2d_data_path'], item['training_3d_data_path'], item['training_label_path'])
g = training_data[0][0]
print(g)
options = {
    'node_size': 100,
    'width': 1,
}
G = dgl.to_networkx(g)
plt.figure(figsize=[15,7])
nx.draw(G, **options)


**NOTE: do not run the cells below as the output contains our training execution logs**

<h3><u>3.4.  Training baseline model (SimplePose) </u></h3>

In [ ]:
from src.simple_pose.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'h36m', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'h36m', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/h36m/Processed/train_2d_poses.npy', training_3d_data_path='datasets/h36m/Processed/train_3d_poses.npy', training_label_path='datasets/h36m/Processed/train_actions.npy', testing_2d_data_path='datasets/h36m/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/h36m/Processed/test_3d_poses.npy', testing_label_path='datasets/h36m/Processed/test_actions.npy', save_path='model_outputs/simple_pose/2024-11-18--03-29-34')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 773/773 [00:08<00:00, 88.80it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 67.90802764892578 | Pose Training Loss: 0.6661983132362366 | Action Training Loss: 1.2881903648376465 | Action Train Label Accuracy: 49.450094009663786


Testing =>: 100%|██████████| 293/293 [00:01<00:00, 207.49it/s]


Epoch: 0 | Total Testing Loss: 10.159862518310547 | Pose Testing Loss: 0.09032389521598816 | Action Testing Loss: 1.1274749040603638 | Action Test Label Accuracy: 44.87938947593126
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 773/773 [00:08<00:00, 87.04it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 1.7584266662597656 | Pose Training Loss: 0.015206225216388702 | Action Training Loss: 0.2378043234348297 | Action Train Label Accuracy: 90.50552938560136


Testing =>: 100%|██████████| 293/293 [00:01<00:00, 202.77it/s]


Epoch: 29 | Total Testing Loss: 1.9219890832901 | Pose Testing Loss: 0.014424951747059822 | Action Testing Loss: 0.479493111371994 | Action Test Label Accuracy: 80.16730707652899


<Figure size 640x480 with 0 Axes>

<h3><u>3.5.  Training Graph Convolutional model (SimplePoseGNN) </u></h3>

In [ ]:
from src.simple_pose_gnn.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'h36m', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'h36m', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gnn', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/h36m/Processed/train_2d_poses.npy', training_3d_data_path='datasets/h36m/Processed/train_3d_poses.npy', training_label_path='datasets/h36m/Processed/train_actions.npy', testing_2d_data_path='datasets/h36m/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/h36m/Processed/test_3d_poses.npy', testing_label_path='datasets/h36m/Processed/test_actions.npy', save_path='model_outputs/simple_pose_gnn/2024-11-18--03-35-29')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 773/773 [02:07<00:00,  6.08it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 14.24456787109375 | Pose Training Loss: 0.1294834166765213 | Action Training Loss: 1.2962241172790527 | Action Train Label Accuracy: 47.89842912884378


Testing =>: 100%|██████████| 293/293 [00:41<00:00,  7.10it/s]


Epoch: 0 | Total Testing Loss: 6.902277946472168 | Pose Testing Loss: 0.056832555681467056 | Action Testing Loss: 1.2190204858779907 | Action Test Label Accuracy: 54.039918881417435
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 773/773 [02:09<00:00,  5.96it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 2.1740164756774902 | Pose Training Loss: 0.018767505884170532 | Action Training Loss: 0.2972647547721863 | Action Train Label Accuracy: 87.96221418029639


Testing =>: 100%|██████████| 293/293 [00:43<00:00,  6.69it/s]


Epoch: 29 | Total Testing Loss: 2.0666732788085938 | Pose Testing Loss: 0.015275339595973492 | Action Testing Loss: 0.5391401052474976 | Action Test Label Accuracy: 76.09136514035649


<Figure size 640x480 with 0 Axes>

<h3><u>3.6.  Training Graph Transformer model (SimplePoseGAT) </u></h3>

In [ ]:
from src.simple_pose_gat.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'h36m', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'h36m', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gat', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/h36m/Processed/train_2d_poses.npy', training_3d_data_path='datasets/h36m/Processed/train_3d_poses.npy', training_label_path='datasets/h36m/Processed/train_actions.npy', testing_2d_data_path='datasets/h36m/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/h36m/Processed/test_3d_poses.npy', testing_label_path='datasets/h36m/Processed/test_actions.npy', save_path='model_outputs/simple_pose_gat/2024-11-18--05-02-06')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 773/773 [03:02<00:00,  4.24it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 7.1949896812438965 | Pose Training Loss: 0.059793420135974884 | Action Training Loss: 1.215651512145996 | Action Train Label Accuracy: 51.03713887147969


Testing =>: 100%|██████████| 293/293 [01:07<00:00,  4.36it/s]


Epoch: 0 | Total Testing Loss: 3.8429837226867676 | Pose Testing Loss: 0.02562158741056919 | Action Testing Loss: 1.280826210975647 | Action Test Label Accuracy: 39.145052833813644
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 773/773 [03:02<00:00,  4.24it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 0.9923760890960693 | Pose Training Loss: 0.007883653976023197 | Action Training Loss: 0.20401187241077423 | Action Train Label Accuracy: 91.93841861593515


Testing =>: 100%|██████████| 293/293 [01:07<00:00,  4.33it/s]


Epoch: 29 | Total Testing Loss: 1.6089502573013306 | Pose Testing Loss: 0.00904751941561699 | Action Testing Loss: 0.7041975259780884 | Action Test Label Accuracy: 73.60977692389797


<Figure size 640x480 with 0 Axes>

<h3><u>3.7.  Training Toplogy Adaptive Graph Convolutional model (SimplePoseTAG) </u></h3>

In [ ]:
from src.simple_pose_tag.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'h36m', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'h36m', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'h36m', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_tag', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/h36m/Processed/train_2d_poses.npy', training_3d_data_path='datasets/h36m/Processed/train_3d_poses.npy', training_label_path='datasets/h36m/Processed/train_actions.npy', testing_2d_data_path='datasets/h36m/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/h36m/Processed/test_3d_poses.npy', testing_label_path='datasets/h36m/Processed/test_actions.npy', save_path='model_outputs/simple_pose_tag/2024-11-18--07-07-28')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 773/773 [02:32<00:00,  5.06it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 12.531847953796387 | Pose Training Loss: 0.11242948472499847 | Action Training Loss: 1.288909673690796 | Action Train Label Accuracy: 47.1888077957261


Testing =>: 100%|██████████| 293/293 [00:49<00:00,  5.97it/s]


Epoch: 0 | Total Testing Loss: 5.952681064605713 | Pose Testing Loss: 0.04794268682599068 | Action Testing Loss: 1.1584112644195557 | Action Test Label Accuracy: 59.213896894012166
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 773/773 [02:38<00:00,  4.89it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 1.3589017391204834 | Pose Training Loss: 0.012065859511494637 | Action Training Loss: 0.15231357514858246 | Action Train Label Accuracy: 94.35234417645513


Testing =>: 100%|██████████| 293/293 [00:48<00:00,  6.02it/s]


Epoch: 29 | Total Testing Loss: 1.4310086965560913 | Pose Testing Loss: 0.0076921056024730206 | Action Testing Loss: 0.6617979407310486 | Action Test Label Accuracy: 80.23801899882591


<Figure size 640x480 with 0 Axes>

<h2><u>4. Custom Dataset</u></h2>


<h3><u>4.1. Summary of the dataset </u></h3>

Alongside the Human 3.6M dataset, we created a custom dataset containing 34,594 human poses. These poses were recorded from a single individual performing five distinct actions, captured from ten different camera angles.

Instead of utilizing a 3D motion capture system, we recorded videos using a digital camera equipped with a global shutter. The 2D and 3D poses were labeled using MediaPipe.

To introduce natural variability, we allowed the individual to incorporate turns and movements while performing the actions, ensuring the dataset reflects more realistic motion patterns.

<h3><u>4.2 Preparing the dataset</u></h3>

We selected five distinct actions for capture: "arm_stretch," "leg_stretch," "lunges," "side_stretch," and "walking." These actions were recorded using a camera mounted on a stand with a global shutter to eliminate motion blur. The recordings were taken from various camera positions and angles to introduce variability.

Here are the steps to record and prepare your custom dataset:

1. Use the Python script `data_collection/record_video.py` to record the actions and save the videos in MP4 format.
2. Use the Python script `data_collection/extract_poses.py` to extract 2D and 3D poses with MediaPipe, which automatically labels the poses, and save the poses in Numpy format. Ensure the action names and video index filenames are updated correctly.

We included a sample video located at `data_collection/sample.mp4`. Running the script `data_collection/check_video.py` will display the following clip: the left side shows the original video, while the right side displays the annotated version with poses marked.


<div align="center">
<img alt="Sample Mediapose Output" width="100%" src="data_collection/sample.gif">
</div>



<h3><u>4.3 Visualizing poses </u></h3>


Let's visualise the poses for the 5 actions in the custom dataset

In [ ]:
from utils.visualization_utils import visualize_2d_pose_actions_custom

out_poses_2d = np.load("datasets/custom/Processed/test_2d_poses.npy")
out_actions = np.load("datasets/custom/Processed/test_actions.npy")

for action in np.unique(out_actions):
    out_poses_2d_action = out_poses_2d[out_actions == action]
    visualize_2d_pose_actions_custom(out_poses_2d_action, action=action)

Let's visualise the first 2D pose and corresponding 3D poses in the custom dataset

In [ ]:
import numpy as np

from utils.visualization_utils import visualize_2d_pose_custom, visualize_3d_pose_custom

poses_2d = np.load("datasets/custom/Processed/test_2d_poses.npy")
poses_3d = np.load("datasets/custom/Processed/test_3d_poses.npy")

visualize_2d_pose_custom(poses_2d[0])
visualize_3d_pose_custom(poses_3d[0], elev=110, azim=90)

Let's visualise the distribution of the 5 actions in the custom train and test datasets

In [ ]:
train_actions = np.load("datasets/custom/Processed/train_actions.npy")
show_action_stats(train_actions, "Train")

In [ ]:
test_actions = np.load("datasets/custom/Processed/test_actions.npy")
show_action_stats(test_actions, "Test")

<h3><u>4.4. Training baseline model (SimplePose)</u></h3>

In [ ]:
from src.simple_pose.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_custom', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/custom/Processed/train_2d_poses.npy', training_3d_data_path='datasets/custom/Processed/train_3d_poses.npy', training_label_path='datasets/custom/Processed/train_actions.npy', testing_2d_data_path='datasets/custom/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/custom/Processed/test_3d_poses.npy', testing_label_path='datasets/custom/Processed/test_actions.npy', save_path='model_outputs/simple_pose_custom/2024-11-17--09-21-41')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 126/126 [00:01<00:00, 82.78it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 158.8680877685547 | Pose Training Loss: 1.5735925436019897 | Action Training Loss: 1.508824110031128 | Action Train Label Accuracy: 37.423122320929366


Testing =>: 100%|██████████| 10/10 [00:00<00:00, 170.78it/s]


Epoch: 0 | Total Testing Loss: 14.504740715026855 | Pose Testing Loss: 0.11845336109399796 | Action Testing Loss: 2.659405469894409 | Action Test Label Accuracy: 20.0
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 126/126 [00:01<00:00, 81.09it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 3.5527873039245605 | Pose Training Loss: 0.03356274217367172 | Action Training Loss: 0.19651304185390472 | Action Train Label Accuracy: 93.42734671056718


Testing =>: 100%|██████████| 10/10 [00:00<00:00, 177.21it/s]


Epoch: 29 | Total Testing Loss: 3.8463234901428223 | Pose Testing Loss: 0.02288799360394478 | Action Testing Loss: 1.55752432346344 | Action Test Label Accuracy: 56.208333333333336


<Figure size 640x480 with 0 Axes>

<h3><u>4.5. Training Graph Convolutional model (SimplePoseGNN)</u></h3>

In [ ]:
from src.simple_pose_gnn.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gnn_custom', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/custom/Processed/train_2d_poses.npy', training_3d_data_path='datasets/custom/Processed/train_3d_poses.npy', training_label_path='datasets/custom/Processed/train_actions.npy', testing_2d_data_path='datasets/custom/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/custom/Processed/test_3d_poses.npy', testing_label_path='datasets/custom/Processed/test_actions.npy', save_path='model_outputs/simple_pose_gnn_custom/2024-11-17--08-39-44')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 126/126 [00:22<00:00,  5.63it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 37.714317321777344 | Pose Training Loss: 0.36073991656303406 | Action Training Loss: 1.6403099298477173 | Action Train Label Accuracy: 21.74007579051997


Testing =>: 100%|██████████| 10/10 [00:01<00:00,  6.88it/s]


...

Training =>: 100%|██████████| 126/126 [00:21<00:00,  5.85it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 2.6875386238098145 | Pose Training Loss: 0.02336752787232399 | Action Training Loss: 0.35078567266464233 | Action Train Label Accuracy: 86.81120705721564


Testing =>: 100%|██████████| 10/10 [00:01<00:00,  5.61it/s]


Epoch: 29 | Total Testing Loss: 1.0730615854263306 | Pose Testing Loss: 0.00841727014631033 | Action Testing Loss: 0.23133455216884613 | Action Test Label Accuracy: 95.75


<Figure size 640x480 with 0 Axes>

<h3><u>4.6. Training Graph Transformer model (SimplePoseGAT)</u></h3>

In [ ]:
from src.simple_pose_gat.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gat_custom', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/custom/Processed/train_2d_poses.npy', training_3d_data_path='datasets/custom/Processed/train_3d_poses.npy', training_label_path='datasets/custom/Processed/train_actions.npy', testing_2d_data_path='datasets/custom/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/custom/Processed/test_3d_poses.npy', testing_label_path='datasets/custom/Processed/test_actions.npy', save_path='model_outputs/simple_pose_gat_custom/2024-11-17--09-05-38')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 126/126 [00:29<00:00,  4.29it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 18.847429275512695 | Pose Training Loss: 0.1733040064573288 | Action Training Loss: 1.5170223712921143 | Action Train Label Accuracy: 27.290799527862337


Testing =>: 100%|██████████| 10/10 [00:02<00:00,  4.56it/s]


Epoch: 0 | Total Testing Loss: 4.888330936431885 | Pose Testing Loss: 0.03508218750357628 | Action Testing Loss: 1.3801120519638062 | Action Test Label Accuracy: 34.708333333333336
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 126/126 [00:27<00:00,  4.62it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 0.4308996796607971 | Pose Training Loss: 0.003506248816847801 | Action Training Loss: 0.08027482032775879 | Action Train Label Accuracy: 97.85053115487358


Testing =>: 100%|██████████| 10/10 [00:02<00:00,  3.98it/s]


Epoch: 29 | Total Testing Loss: 0.3450351655483246 | Pose Testing Loss: 0.00305281113833189 | Action Testing Loss: 0.03975405916571617 | Action Test Label Accuracy: 99.5


<Figure size 640x480 with 0 Axes>

<h3><u>4.7. Training Toplogy Adaptive Graph Convolutional model (SimplePoseTAG)</u></h3>

In [ ]:
from src.simple_pose_tag.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 1,
    'pose_loss_multiplier': 100,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_tag_custom', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

Training args are: Namespace(learning_rate=3e-05, num_epochs=30, batch_size=256, action_loss_multiplier=1, pose_loss_multiplier=100, training_2d_data_path='datasets/custom/Processed/train_2d_poses.npy', training_3d_data_path='datasets/custom/Processed/train_3d_poses.npy', training_label_path='datasets/custom/Processed/train_actions.npy', testing_2d_data_path='datasets/custom/Processed/test_2d_poses.npy', testing_3d_data_path='datasets/custom/Processed/test_3d_poses.npy', testing_label_path='datasets/custom/Processed/test_actions.npy', save_path='model_outputs/simple_pose_tag_custom/2024-11-17--08-51-33')
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 126/126 [00:24<00:00,  5.16it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 24.604963302612305 | Pose Training Loss: 0.22997219860553741 | Action Training Loss: 1.6077426671981812 | Action Train Label Accuracy: 24.485929055103437


Testing =>: 100%|██████████| 10/10 [00:01<00:00,  5.55it/s]


Epoch: 0 | Total Testing Loss: 5.861079216003418 | Pose Testing Loss: 0.04376217722892761 | Action Testing Loss: 1.4848613739013672 | Action Test Label Accuracy: 40.0
Starting EPOCH: 2 / 30


...

Training =>: 100%|██████████| 126/126 [00:24<00:00,  5.08it/s]


Saving at epoch 29
Epoch: 29 | Total Training Loss: 1.752930760383606 | Pose Training Loss: 0.016896378248929977 | Action Training Loss: 0.06329337507486343 | Action Train Label Accuracy: 98.77616947257253


Testing =>: 100%|██████████| 10/10 [00:02<00:00,  4.76it/s]


Epoch: 29 | Total Testing Loss: 0.5635214447975159 | Pose Testing Loss: 0.005169626325368881 | Action Testing Loss: 0.04655873775482178 | Action Test Label Accuracy: 98.08333333333333


<Figure size 640x480 with 0 Axes>

<h2><u>5. Evaluation </u></h2>

In this section we will be showing the training and testing graphs for each model across both datasets along with confusion matrices for each model. 

Before you run the cells below, you will need to download the final_weights.zip from our Google Drive Link.

**Please run all the cells in this section to generate the graphs and the confusion matrices. We use the same image names for our explanation below**

To evaluate our models, we have decided to use these metrics.

1. Pose Loss: To measure the pose loss per epoch using Mean Squared Loss
2. Label Loss: To measure the label loss per epoch using Cross Entropy Loss
3. Total Loss: To measure our multitasking loss per epoch using this formula: pose_loss_multiplier * pose_loss + action_loss_multiplier * action_loss where pose_loss_multiplier = 100 and action_loss_multiplier = 1
4. Accuracies: To measure the number of right label predictions per epoch

<h3><u>5.1 Human 3.6M Dataset </u></h3>

In [2]:

# Prepare the loss and accuracies graphs
import os
from utils.visualization_utils import save_fig

TESTING_2D_DATA_PATH = os.path.join('datasets', 'h36m', 'Processed', 'test_2d_poses.npy')
TESTING_3D_DATA_PATH =  os.path.join('datasets', 'h36m', 'Processed', 'test_3d_poses.npy')
TESTING_LABEL_PATH = os.path.join('datasets', 'h36m', 'Processed', 'test_actions.npy')
SAVE_PATH = os.path.join('visualizations', 'graphs', 'h36m')

save_files = {
    'SimplePose': os.path.join('final_weights', 'h36m', 'simple_pose_h36m.pth'),
    'SimplePoseGNN': os.path.join('final_weights', 'h36m', "simple_pose_gnn_h36m.pth"),
    'SimplePoseGAT': os.path.join('final_weights', 'h36m', "simple_pose_gat_h36m.pth"), 
    'SimplePoseTAG': os.path.join('final_weights', 'h36m', "simple_pose_tag_h36m.pth"),
}

save_fig(save_files, 'h36m', SAVE_PATH)


from src.simple_pose.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePose'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

from src.simple_pose_gnn.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePoseGNN'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

from src.simple_pose_gat.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePoseGAT'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

from src.simple_pose_tag.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePoseTAG'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

Testing =>: 100%|██████████| 293/293 [00:01<00:00, 235.87it/s]


Confusion Matrix Without Normalization


Testing =>: 100%|██████████| 293/293 [01:35<00:00,  3.07it/s]


Confusion Matrix Without Normalization


Testing =>: 100%|██████████| 293/293 [01:48<00:00,  2.70it/s]


Confusion Matrix Without Normalization


Testing =>: 100%|██████████| 293/293 [01:35<00:00,  3.06it/s]


Confusion Matrix Without Normalization


<h4>Accuracies and Confusion Matrices</h4>

<h5>Human 3.6M Confusion Matrix</h5>


| SimplePose| SimplePoseGNN |
|---------------|---------------|
|![SimplePose](./visualizations/graphs/h36m/SimplePose_conf_matrix.png)|![SimplePoseGNN](./visualizations/graphs/h36m/SimplePoseGNN_conf_matrix.png)|
|SimplePoseGAT| SimplePoseTAG|
| ![SimplePoseGAT](./visualizations/graphs/h36m/SimplePoseGAT_conf_matrix.png) | ![SimplePoseTAG](./visualizations/graphs/h36m/SimplePoseTAG_conf_matrix.png) |


<h5>Human 3.6M Accuracies</h5>

| All Models Training Accuracies | All Models Testing Accuracies |
|---------------|---------------|
|![All Models Training Accuracies ](./visualizations/graphs/h36m/h36m_training_accuracies.png)|![All Models Testing Accuracies](./visualizations/graphs/h36m/h36m_testing_accuracies.png)|


<h3><u>5.2 Custom Dataset </u></h3>

In [3]:

# Prepare the loss and accuracies graphs
import os
from utils.visualization_utils import save_fig

TESTING_2D_DATA_PATH = os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy')
TESTING_3D_DATA_PATH =  os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy')
TESTING_LABEL_PATH = os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy')
SAVE_PATH = os.path.join('visualizations', 'graphs', 'custom')

save_files = {
    'SimplePose': os.path.join('final_weights', 'custom', 'simple_pose_custom.pth'),
    'SimplePoseGNN': os.path.join('final_weights', 'custom', "simple_pose_gnn_custom.pth"),
    'SimplePoseGAT': os.path.join('final_weights', 'custom', "simple_pose_gat_custom.pth"), 
    'SimplePoseTAG': os.path.join('final_weights', 'custom', "simple_pose_tag_custom.pth"),
}

save_fig(save_files, 'custom', SAVE_PATH)


from src.simple_pose.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePose'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

from src.simple_pose_gnn.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePoseGNN'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

from src.simple_pose_gat.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePoseGAT'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

from src.simple_pose_tag.train_and_test import generate_confusion_matrix
generate_confusion_matrix(
    weights_path=save_files['SimplePoseTAG'],
    save_path=SAVE_PATH,
    testing_2d_path= TESTING_2D_DATA_PATH,
    testing_3d_path= TESTING_3D_DATA_PATH,
    testing_label_path= TESTING_LABEL_PATH
)

Testing =>: 100%|██████████| 10/10 [00:00<00:00, 82.18it/s]


Confusion Matrix Without Normalization


Testing =>: 100%|██████████| 10/10 [00:03<00:00,  2.89it/s]


Confusion Matrix Without Normalization


Testing =>: 100%|██████████| 10/10 [00:04<00:00,  2.43it/s]


Confusion Matrix Without Normalization


Testing =>: 100%|██████████| 10/10 [00:06<00:00,  1.55it/s]


Confusion Matrix Without Normalization


<h2><u>6. Lessons Learnt & Conclusions </u></h2>


Talk about the history of development and optimizations we made

<h2><u>7. Resources</u></h2>


TODO add blogs, documentations , us